In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import os
import csv
import datetime
import moment

# import api key
from config import api_key
   
# Weather URL
#History weather bulk for Detroit from January 01, 2014 to December 31, 2018
weather_url = "http://history.openweathermap.org//storage/dc1e5df660e33aec203660c9b2000bc3.json"
solunar_url = "https://api.solunar.org/solunar/latitude,longitude,date,tz"
#https://api.solunar.org/solunar/42.66,-84.07,20180207,-4

# make request and store response
response = requests.get(weather_url)
response_2 = requests.get(solunar_url)

# print status code
print(response.status_code)
print(response_2.status_code)

200
200


In [2]:
#JSONify response
response_json = response.json()
response_2_json = response_2.json()

In [3]:
# print first article
print(f"The first response is {json.dumps(response_json[0], indent=4)}.")

The first response is {
    "city_id": 4990729,
    "main": {
        "temp": 265.33,
        "temp_min": 263.71,
        "temp_max": 266.48,
        "pressure": 1025,
        "humidity": 61
    },
    "wind": {
        "speed": 5,
        "deg": 250
    },
    "clouds": {
        "all": 90
    },
    "weather": [
        {
            "id": 804,
            "main": "Clouds",
            "description": "overcast clouds",
            "icon": "04"
        }
    ],
    "dt": 1388534400,
    "dt_iso": "2014-01-01 00:00:00 +0000 UTC"
}.


In [4]:
# count of responses
print(f"We received {len(response_json)} responses.") 

We received 33454 responses.


In [5]:
# reference to CSV file
DET_csv = os.path.join("City Data","Detroit","detroit_data_fixed.csv")

with open(DET_csv, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=",")
    print(csvreader)

DET_crime_df = pd.read_csv(DET_csv)
DET_crime_df.head()


,Location,Offense,Date,Unix Date
0,"(42.4224, -82.986)",LARCENY: LARCENY (OTHER),01/02/2014,1388638800
1,"(42.386, -83.2736)",ESCAPE: FUGITIVE,01/03/2014,1388725200
2,"(42.4424, -83.0719)",ASSAULT: INTIMIDATION,01/04/2014,1388811600
3,"(42.3373, -83.1961)",LARCENY: LARCENY (OTHER),01/04/2014,1388811600
4,"(42.3314, -83.1239)",LARCENY: LARCENY (OTHER),01/06/2014,1388984400


In [6]:
DET_crime_df["Temp_Max"] = ""
DET_crime_df["Wind_Speed"] = ""
DET_crime_df["Weather_Type"] = ""
DET_crime_df.head()

,Location,Offense,Date,Unix Date,Temp_Max,Wind_Speed,Weather_Type
0,"(42.4224, -82.986)",LARCENY: LARCENY (OTHER),01/02/2014,1388638800,,,
1,"(42.386, -83.2736)",ESCAPE: FUGITIVE,01/03/2014,1388725200,,,
2,"(42.4424, -83.0719)",ASSAULT: INTIMIDATION,01/04/2014,1388811600,,,
3,"(42.3373, -83.1961)",LARCENY: LARCENY (OTHER),01/04/2014,1388811600,,,
4,"(42.3314, -83.1239)",LARCENY: LARCENY (OTHER),01/06/2014,1388984400,,,


In [15]:
# create a params dict that will be updated with weather data each iteration
params = {"key": api_key}
weather_url = "http://history.openweathermap.org//storage/dc1e5df660e33aec203660c9b2000bc3.json"


In [16]:
# Use unix date to identify weather
for index, row in DET_crime_df.iterrows():
    unix_date = row["Unix Date"]

    # change date each iteration while leaving original params in place
    params["keyword"] = unix_date

    # make request
    response = requests.get(weather_url, params=params).json()

    # convert to json
    response_json = response["response_json"]

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        DET_crime_df.loc[index,"Temp_Max"] = response_json["main"]["temp_max"]
        #DET_crime_df.loc[index, "Wind_Speed"] = response_json["wind"]["speed"]
        #DET_crime_df.loc[index, "Weather_Type"] = response_json["weather"][0]["main"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    

TypeError: list indices must be integers or slices, not str